## Imports

In [1]:
from pathlib import Path # reads paths in the current OS
import unicodedata
import pandas as pd
import numpy as np
import json
import re
import yaml
import utils as ut
from stop_words import get_stop_words
from unidecode import unidecode

## Reading

In [2]:
with open('paths.yaml') as file:
    config = yaml.full_load(file)

data         = pd.read_csv(Path(config['input_path']) / "abastiment.csv", sep=",").fillna('')
locations_df = pd.read_csv(Path(config['input_path']) /  'municipis_merge.csv').fillna('')

with open('com_comprar_kw.yaml') as file:
    com_comprar = yaml.full_load(file)
with open('product_list.yaml') as file:
    product_cat = yaml.full_load(file)

In [3]:
data.head()

,PROJECTE,URGENT,PRODUCTE(S),MUNICIPI,COM COMPRAR,OBSERVACIONS,CCPAE,COMARCA
0,3 Fermentats I Un Destí,Sí,1 Pa de Massa Mare Natural de 2kg 2 Vins Natur...,Torelló,Ho repartim a domicili per tot Osona resta de ...,Ens fa il.lusió poder colaborar per oferir-vos...,Sí,Osona
1,30 Cabres,,Formatges de llet crua de cabres que pasturen ...,Bellver de Cerdanya,"Contacta per telefon, mail o xarxes socials (@...",Tots els nostres formatges estan elaborats amb...,,Cerdanya
2,7 de Ribera Hostal-Agrobotiga,Sí,Verdures de producció propia,Móra d'Ebre,A l'agrobotiga dijous i divendres de 10:00 a 1...,www.biosferacomestible.cat,,Ribera d’Ebre
3,A Granel,,"Cosmètica, productes d’higiène i neteja ecològics",Barcelona,Venda directa i en punts de recollida per tota...,,Sí,Barcelona
4,AgroMontserratí - Cooperativa del Montserratí,,"Fruita, verdura, transformats, elaborats, pins...",Olesa de Montserrat,A través del Whatsap i Telèfon - XXXXXXXXX A l...,www.agromontserrati.cat,,Baix Llobregat


In [4]:
locations_df.head()

,Municipi,Comarca,Capital,Provincia
0,Abella de la Conca,Pallars Jussà,Tremp,Lleida
1,Castell de Mur,Pallars Jussà,Tremp,Lleida
2,Conca de Dalt,Pallars Jussà,Tremp,Lleida
3,Gavet de la Conca,Pallars Jussà,Tremp,Lleida
4,Isona i Conca Dellà,Pallars Jussà,Tremp,Lleida


In [5]:
#load a set of stop words
stopwords = get_stop_words('catalan')
#add new stopwords
newStopWords = ['que','des', 'al', 'del', 'ho', 'd', 'l','per','tambe', 'fins',
               'a', 'cap', 'hi', 'ni', 'no']
stopwords.extend(newStopWords)

In [6]:
delivery_patt = ['repart', 'domicil', 'envi', 'recoll', 'dist']

In [7]:
# comarca_new is necessary because it contains some "repartim a..." text
cols_to_extract_locs = ['COM COMPRAR', 'OBSERVACIONS', 'comarca_new']

In [8]:
com_typos = {
    'Al Urgell':'Alt Urgell',
    'Bages-Moianès':'Moianès',
    'Moianes-Bages':'Moianès',
    'Barcelona':'Barcelonès',
    'Maresme-Barcelonès':'Maresme',
    'Tarragona':'Tarragonès',
    'Baix Montseny':'Vallès Oriental',
    'Baixa Cerdanya':'Cerdanya',
    'Vall Aran':"Vall d'Aran",
    'Alt Maresme':'Maresme',
    'Penedès':'Alt Penedès',
    'Lluçanès':'Osona', #should we consider it a comarca?
            }

## Pre-process columns

In [9]:
data['comarca_new'] = data['COMARCA'].replace(com_typos)

In [10]:
data['comarca_new'] = data['comarca_new'].apply(lambda x: ut.check_comarca_spelling(
    x,locations_df['Comarca'],stopwords) if x not in locations_df['Comarca'] else x)

Not found: Repartim al Bages, Solsonès, Barcelonès i Berguedà


In [11]:
data.head()

,PROJECTE,URGENT,PRODUCTE(S),MUNICIPI,COM COMPRAR,OBSERVACIONS,CCPAE,COMARCA,comarca_new
0,3 Fermentats I Un Destí,Sí,1 Pa de Massa Mare Natural de 2kg 2 Vins Natur...,Torelló,Ho repartim a domicili per tot Osona resta de ...,Ens fa il.lusió poder colaborar per oferir-vos...,Sí,Osona,Osona
1,30 Cabres,,Formatges de llet crua de cabres que pasturen ...,Bellver de Cerdanya,"Contacta per telefon, mail o xarxes socials (@...",Tots els nostres formatges estan elaborats amb...,,Cerdanya,Cerdanya
2,7 de Ribera Hostal-Agrobotiga,Sí,Verdures de producció propia,Móra d'Ebre,A l'agrobotiga dijous i divendres de 10:00 a 1...,www.biosferacomestible.cat,,Ribera d’Ebre,Ribera d'Ebre
3,A Granel,,"Cosmètica, productes d’higiène i neteja ecològics",Barcelona,Venda directa i en punts de recollida per tota...,,Sí,Barcelona,Barcelonès
4,AgroMontserratí - Cooperativa del Montserratí,,"Fruita, verdura, transformats, elaborats, pins...",Olesa de Montserrat,A través del Whatsap i Telèfon - XXXXXXXXX A l...,www.agromontserrati.cat,,Baix Llobregat,Baix Llobregat


In [12]:
for col in cols_to_extract_locs+['PRODUCTE(S)']:
    data[col+'_prep'] = data[col].apply(lambda x: ut.pre_process(x, stopwords,sw=True))

for col in ['Municipi', 'Comarca', 'Capital', 'Provincia']:
    locations_df[col+'_prep'] = locations_df[col].apply(lambda x: ut.pre_process(x, stopwords,sw=True))

In [13]:
data.head()

,PROJECTE,URGENT,PRODUCTE(S),MUNICIPI,COM COMPRAR,OBSERVACIONS,CCPAE,COMARCA,comarca_new,COM COMPRAR_prep,OBSERVACIONS_prep,comarca_new_prep,PRODUCTE(S)_prep
0,3 Fermentats I Un Destí,Sí,1 Pa de Massa Mare Natural de 2kg 2 Vins Natur...,Torelló,Ho repartim a domicili per tot Osona resta de ...,Ens fa il.lusió poder colaborar per oferir-vos...,Sí,Osona,Osona,repartim domicili osona resta catalunya consul...,il lusio colaborar oferir vos tres productes f...,osona,pa massa mare natural kg vins naturals formatg...
1,30 Cabres,,Formatges de llet crua de cabres que pasturen ...,Bellver de Cerdanya,"Contacta per telefon, mail o xarxes socials (@...",Tots els nostres formatges estan elaborats amb...,,Cerdanya,Cerdanya,contacta telefon mail xarxes socials cabres mi...,tots nostres formatges elaborats llet crua nos...,cerdanya,formatges llet crua cabres pasturen pirineu
2,7 de Ribera Hostal-Agrobotiga,Sí,Verdures de producció propia,Móra d'Ebre,A l'agrobotiga dijous i divendres de 10:00 a 1...,www.biosferacomestible.cat,,Ribera d’Ebre,Ribera d'Ebre,agrobotiga dijous divendres dijous tarde repar...,www biosferacomestible cat,ribera ebre,verdures produccio propia
3,A Granel,,"Cosmètica, productes d’higiène i neteja ecològics",Barcelona,Venda directa i en punts de recollida per tota...,,Sí,Barcelona,Barcelonès,venda directa punts recollida tota catalunya d...,,barcelones,cosmetica productes higiene neteja ecologics
4,AgroMontserratí - Cooperativa del Montserratí,,"Fruita, verdura, transformats, elaborats, pins...",Olesa de Montserrat,A través del Whatsap i Telèfon - XXXXXXXXX A l...,www.agromontserrati.cat,,Baix Llobregat,Baix Llobregat,traves whatsap telefon xxxxxxxxx agrobotiga c ...,www agromontserrati cat,baix llobregat,fruita verdura transformats elaborats pinsos p...


## New columns

### Locations

In [14]:
for data_field in cols_to_extract_locs:
    for loc in ['Comarca','Capital','Provincia','Municipi']:
        # obtain the locations from the free text fields
        ut.get_text_locations(data, loc.lower(),data_field,locations_df,loc,delivery_patt)

In [15]:
# Dictionary to translate municipis to comarca
mun_to_com_dict = locations_df[locations_df['Municipi']!=''].set_index('Municipi')['Comarca'].to_dict()

In [16]:
data[['capital','municipi']] = data[['capital','municipi']].replace(mun_to_com_dict,regex=True)
data['all_comarques']        = (data['capital']+','+data['municipi']+','+data['comarca']
                               ).str.strip(',').str.split(',')
data.drop(['capital','municipi', 'comarca_new_prep','comarca_new','comarca'],axis=1,inplace=True)

In [17]:
data['all_comarques'] = data['all_comarques'].apply(lambda x: ','.join(set(x)))
data['all_comarques'] = data['all_comarques'].str.replace(r'\bCatalunya\b','Tota Catalunya')

In [18]:
data.head()

,PROJECTE,URGENT,PRODUCTE(S),MUNICIPI,COM COMPRAR,OBSERVACIONS,CCPAE,COMARCA,COM COMPRAR_prep,OBSERVACIONS_prep,PRODUCTE(S)_prep,provincia,all_comarques
0,3 Fermentats I Un Destí,Sí,1 Pa de Massa Mare Natural de 2kg 2 Vins Natur...,Torelló,Ho repartim a domicili per tot Osona resta de ...,Ens fa il.lusió poder colaborar per oferir-vos...,Sí,Osona,repartim domicili osona resta catalunya consul...,il lusio colaborar oferir vos tres productes f...,pa massa mare natural kg vins naturals formatg...,,"Osona,Solsonès,Tota Catalunya"
1,30 Cabres,,Formatges de llet crua de cabres que pasturen ...,Bellver de Cerdanya,"Contacta per telefon, mail o xarxes socials (@...",Tots els nostres formatges estan elaborats amb...,,Cerdanya,contacta telefon mail xarxes socials cabres mi...,tots nostres formatges elaborats llet crua nos...,formatges llet crua cabres pasturen pirineu,,Cerdanya
2,7 de Ribera Hostal-Agrobotiga,Sí,Verdures de producció propia,Móra d'Ebre,A l'agrobotiga dijous i divendres de 10:00 a 1...,www.biosferacomestible.cat,,Ribera d’Ebre,agrobotiga dijous divendres dijous tarde repar...,www biosferacomestible cat,verdures produccio propia,,Ribera d'Ebre
3,A Granel,,"Cosmètica, productes d’higiène i neteja ecològics",Barcelona,Venda directa i en punts de recollida per tota...,,Sí,Barcelona,venda directa punts recollida tota catalunya d...,,cosmetica productes higiene neteja ecologics,,"Barcelonès,Tota Catalunya"
4,AgroMontserratí - Cooperativa del Montserratí,,"Fruita, verdura, transformats, elaborats, pins...",Olesa de Montserrat,A través del Whatsap i Telèfon - XXXXXXXXX A l...,www.agromontserrati.cat,,Baix Llobregat,traves whatsap telefon xxxxxxxxx agrobotiga c ...,www agromontserrati cat,fruita verdura transformats elaborats pinsos p...,,Baix Llobregat


In [19]:
data['all_comarques']

0      Osona,Solsonès,Tota Catalunya
1                           Cerdanya
2                      Ribera d'Ebre
3          Barcelonès,Tota Catalunya
4                     Baix Llobregat
                   ...              
221                   Baix Llobregat
222                          Maresme
223                         Ripollès
224                            Osona
225                  Vallès Oriental
Name: all_comarques, Length: 226, dtype: object

### Binary variables

In [20]:
# create binary variables representing whether they have a type of product (1) or not
for key, val in product_cat.items():
    data[key]=0
    data.loc[data['PRODUCTE(S)'+'_prep'].str.contains(r'\b'+r'\b|\b'.join(val)+r'\b'),key] = 1
    
# create binary variables representing whether they have a type payment method, contact info... (1) or not
for key, val in com_comprar.items():
    data[key]=0
    data.loc[data['COM COMPRAR'+'_prep'].str.contains(r'\b'+r'\b|\b'.join(val)+r'\b'),key] = 1
# improve the website and social network searches adding another column
data.loc[(data['web']!=1) & (data['OBSERVACIONS'+'_prep'].str.contains(
    r'\b'+r'\b|\b'.join(com_comprar['web'])+r'\b')),'web'] = 1
data.loc[(data['socialnet']!=1) & (data['OBSERVACIONS'+'_prep'].str.contains(
    r'\b'+r'\b|\b'.join(com_comprar['socialnet'])+r'\b')),'socialnet'] = 1

### Number of type of products

In [21]:
# Creating variables about number of products sold:
data['n_prod_princ'] = data['meat'] + data['fruit'] + data['vegetables']
data['n_prod_others'] = data['flowers'] + data['legumes'] + data['mushrooms'] + data['rice'] +\
    data['flour_cereals'] + data['oil_olives_vinager'] + data['eggs'] + data['dairies'] +\
    data['herbs_spices'] + data['hygiene_medicines'] + data['alcohol'] +\
    data['fruit_veggies_products'] + data['drinks'] + data['bread_pastries'] +\
    data['pasta'] + data['others']
data['n_prod_tot'] = data['n_prod_princ'] + data['n_prod_others']

## Save

In [22]:
data.drop(['PRODUCTE(S)_prep','OBSERVACIONS_prep','COM COMPRAR_prep'],axis=1
         ).to_csv(Path(config['input_path']) / 'abastiment_new.csv', index=False)